In [ ]:
import typing
from automaticTB.examples.structures import get_perovskite_structure
from automaticTB.interaction import InteractionEquation
from automaticTB.atomic_orbitals import AO
from automaticTB.printing import print_ao_pairs, print_matrix, print_InteractionPairs
from automaticTB.examples.Perovskite.ao_interaction import (
    get_interaction_values_from_list_AOpairs, get_overlap_values_from_list_AOpairs
)

from automaticTB.functions import (
    get_namedLCs_from_nncluster, 
    get_free_AOpairs_from_nncluster_and_namedLCs
)

structure = get_perovskite_structure()
print("Solving for the free nearest neighbor interaction in Perovskite")
print("Starting ...")

equation_systems: typing.List[InteractionEquation] = []

for i, nncluster in enumerate(structure.nnclusters):
    print("Cluster centered on " + str(nncluster.crystalsites[nncluster.origin_index]))
    print(nncluster.sitesymmetrygroup.groupname)

    named_lcs = get_namedLCs_from_nncluster(nncluster)
    for nlc in named_lcs:
        print(f"\"{nlc.name}\",")
    #    print(nlc.lc)
    free_pairs = get_free_AOpairs_from_nncluster_and_namedLCs(nncluster, named_lcs)
    print("Solve Interaction ...")
    equation = InteractionEquation.from_nncluster_namedLC(nncluster, named_lcs)

    print_ao_pairs(nncluster, equation.free_AOpairs)
    equation_systems.append(equation)


In [ ]:
from automaticTB.interaction import CombinedInteractionEquation

combined_equation = CombinedInteractionEquation(equation_systems)
for i, aopair in enumerate(combined_equation.free_AOpairs):
    print(f"Free pair {i+1}")
    print(aopair)

In [ ]:
from automaticTB.interaction import InteractionPairs
from automaticTB.tightbinding import TightBindingModel, gather_InteractionPairs_into_HijRs, gather_InteractionPairs_into_SijRs

all_interaction_pairs: typing.List[InteractionPairs] = []
all_overlaps_pairs: typing.List[InteractionPairs] = []

# we build the Hamiltonian and Overlap matrix
for equation in equation_systems:
    free_pairs = equation.free_AOpairs
        
    hamiltonian_values = get_interaction_values_from_list_AOpairs(structure.cell, structure.positions, free_pairs)
    overlap_values = get_overlap_values_from_list_AOpairs(free_pairs)

    cluster_interactions = equation.solve_interactions_to_InteractionPairs(hamiltonian_values)
    cluster_overlap = equation.solve_interactions_to_InteractionPairs(overlap_values)

    all_interaction_pairs.append(cluster_interactions)
    all_overlaps_pairs.append(cluster_overlap)

hijRs = gather_InteractionPairs_into_HijRs(all_interaction_pairs)
SijRs = gather_InteractionPairs_into_SijRs(all_overlaps_pairs)

perovskiet_TBmodel = TightBindingModel(structure.cell, structure.positions, structure.types, hijRs, SijRs)

In [ ]:
from automaticTB.examples.kpaths import cubic_kpath
from automaticTB.properties.bandstructure import BandStructureResult
import os

filename = "HalidePerovskite.pdf"

print("Plot the bandstructure")
band_result = BandStructureResult.from_tightbinding_and_kpath(perovskiet_TBmodel, cubic_kpath)
band_result.plot_data(filename)

os.system(f"rm {filename}") # delete the temporary file

In [ ]:
from automaticTB.properties.dos import get_tetrados_result
import numpy as np

tetrados = get_tetrados_result(perovskiet_TBmodel, ngrid=np.array([10,10,10]))
tetrados.plot_data(filename)
os.system(f"rm {filename}")